# 1. 获取日行情数据


In [1]:
# 1. 连接万得服务
from WindPy import w

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒  
w.isconnected() # 判断WindPy是否已经登录成功
# w.stop() # 当需要停止WindPy时，可以使用该命令
          # 注： w.start不重复启动，若需要改变参数，如超时时间，用户可以使用w.stop命令先停止后再启动。
          # 退出时，会自动执行w.stop()，一般用户并不需要执行w.stop 

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [ ]:
date = '2022-09-20'
# all_a = w.wset("SectorConstituent",date = date ,sector=u"全部A股", usedf = True)
sw_index=w.wset("sectorconstituent","date=2018-06-12;sectorid=a39901011g000000;field=wind_code,sec_name",usedf=True)
stock_codes = w.wset("sectorconstituent", "date=2018-06-12;windcode=000300.SH;field=wind_code,sec_name")
pass
